In [8]:
pip install osmnx networkx folium

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\Rutuja\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import osmnx as ox
import networkx as nx
import folium
import heapq

# Configure OSMnx settings
ox.settings.timeout = 300
ox.settings.overpass_endpoint = "https://overpass.kumi.systems/api/interpreter"

def get_coordinates(place_name):
    """Get latitude and longitude for a place name."""
    try:
        return ox.geocode(place_name)
    except Exception as e:
        print(f"Error: Could not find coordinates for '{place_name}'. Please check the name and try again.")
        return None

def dijkstra_manual(graph, start_node, end_node):
    """Manually implement Dijkstra's algorithm to find the shortest path."""
    distances = {node: float('inf') for node in graph.nodes}
    distances[start_node] = 0
    previous_nodes = {node: None for node in graph.nodes}
    priority_queue = [(0, start_node)]

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)

        if current_node == end_node:
            break

        if current_distance > distances[current_node]:
            continue

        for neighbor, edge_data in graph[current_node].items():
            weight = edge_data[0].get('length', 1)  # Use 'length' as the weight
            distance = current_distance + weight

            if distance < distances[neighbor]:
                distances[neighbor] = distance
                previous_nodes[neighbor] = current_node
                heapq.heappush(priority_queue, (distance, neighbor))

    # Reconstruct the shortest path
    path = []
    current = end_node
    while current is not None:
        path.append(current)
        current = previous_nodes[current]
    path.reverse()

    return path

def visualize_route(graph, shortest_path, start_point, end_point):
    """Visualize the route on a map using Folium."""
    route_coordinates = [(graph.nodes[node]['y'], graph.nodes[node]['x']) for node in shortest_path]

    # Create a map centered at the start point
    m = folium.Map(location=start_point, zoom_start=13)

    # Add markers for the start and end points
    folium.Marker(
        location=start_point,
        popup="Start Point",
        icon=folium.Icon(color="green")
    ).add_to(m)

    folium.Marker(
        location=end_point,
        popup="End Point",
        icon=folium.Icon(color="red")
    ).add_to(m)

    # Draw the route on the map
    folium.PolyLine(
        locations=route_coordinates,
        color="blue",
        weight=5,
        opacity=0.7
    ).add_to(m)

    # Save and display the map
    m.save("shortest_route.html")
    print("Map saved as 'shortest_route.html'. Open this file in your browser to view the route.")
    return m

def main():
    # Get user input for start and end points
    start_place = input("Enter the starting point (e.g., Pune, Maharashtra): ")
    end_place = input("Enter the ending point (e.g., Mumbai, Maharashtra): ")

    # Get coordinates for start and end points
    start_point = get_coordinates(start_place)
    end_point = get_coordinates(end_place)

    if start_point is None or end_point is None:
        print("Exiting due to invalid input.")
        return

    # Fetch the road network for a smaller area (to avoid large data fetches)
    print("Fetching road network data...")
    try:
        # Use the start point to fetch a smaller graph (e.g., within 5 km)
        graph = ox.graph_from_point(start_point, dist=5000, network_type='drive')
    except Exception as e:
        print(f"Failed to fetch road network data: {e}")
        return

    # Find the nearest nodes to the start and end points
    start_node = ox.distance.nearest_nodes(graph, start_point[1], start_point[0])
    end_node = ox.distance.nearest_nodes(graph, end_point[1], end_point[0])

    # Get the shortest path using the manual Dijkstra implementation
    print("Calculating the shortest path using Dijkstra's algorithm...")
    shortest_path = dijkstra_manual(graph, start_node, end_node)

    # Visualize the route on a map
    print("Visualizing the route...")
    visualize_route(graph, shortest_path, start_point, end_point)

if __name__ == "__main__":
    main()

Enter the starting point (e.g., Pune, Maharashtra):  Pune, Maharashtra
Enter the ending point (e.g., Mumbai, Maharashtra):  Mumbai, Maharashtra


Fetching road network data...
Calculating the shortest path using Dijkstra's algorithm...
Visualizing the route...
Map saved as 'shortest_route.html'. Open this file in your browser to view the route.
